In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)
filename_grokKey = "..\\GroqAPIKey.txt"
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### Agents with Memory

In [4]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq


In [5]:
#llm= ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
llm = ChatGroq(model="llama-3.1-8b-instant")

In [6]:
search = DuckDuckGoSearchResults()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [7]:
prompt = ChatPromptTemplate(
    [
        (
            "system","You are a helpful assistant. For answering the user query, look for information using DuckDuckGo Search and Wikipedia and then give the final answer."
        ),
        ("placeholder","{history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}"), # must add when using tool calling agent
    ]
)

In [8]:
tools = [search,wikipedia]

In [9]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [10]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True, input_keys=["input"])

In [11]:
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] =  ChatMessageHistory()
    return store[session_id]

In [12]:
agent_with_history = RunnableWithMessageHistory(
    agent_executor, 
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [13]:
print(agent_with_history.input_messages_key)

input


In [14]:
print(agent_with_history.history_messages_key)

history


In [16]:
agent_with_history.invoke(
    {"input":"When was Nelson Mandela born?"},
    config={"configurable":{"session_id":"sess1"}},
    )



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela birthdate'}`


Page: Pablo Sarino
Summary: Pablo Gomez Sarino (June 29, 1911 - December 23, 1987) was a Filipino politician who served as the municipal mayor of Bacoor, Cavite. He previously served as a municipal councilor and later became elected as the mayor in 1959. His terms of office in total made him the longest serving mayor of Bacoor to date.

Page: Dominic Waghorn
Summary: Dominic David Waghorn (born 1968, Lambeth), is a British journalist who is the Diplomatic Editor of Sky News and presenter of the channel's weekly international affairs analysis programme World View.  He was before that US Correspondent of Sky News, the 24-hour television news service operated by Sky Television, part of British Sky Broadcasting. He is based at Sky News' Washington Bureau. He was formerly Sky News' Asia Correspondent, based in Beijing and Middle East Correspondent, based in Jerusalem. He became Sky 

{'input': 'When was Nelson Mandela born?',
 'history': [HumanMessage(content='When was Nelson Mandela born?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Unfortunately, I was unable to find the birthdate of Nelson Mandela.', additional_kwargs={}, response_metadata={})],
 'output': 'Nelson Mandela was born on July 18, 1918.'}

In [17]:
result = agent_with_history.invoke(
    {"input":"What are some of his famous quotes?"},
    config={"configurable":{"session_id":"sess1"}},
    )



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela famous quotes'}`


Page: Ndaba Mandela
Summary: Ndaba Thembekile Zweliyajika Mandela (born 23 December 1982) is an author, mentor, spokesperson, entrepreneur, political consultant, and the grandson of Nelson Mandela. He was born in Soweto, South Africa.
Mandela is currently the co-founder and chairman of the Mandela Institute for Humanity.
He is active in keeping the legacy of his grandfather alive, and introducing Nelson Mandela to a younger generation. He is the co-founder and chairman of Africa Rising Foundation.  He is also the founder of the Mandela Project. He was part of the team for the Mandela Centenary in 2018.
From 2014 - 2021 Ndaba was Vice President of the Pan-African Youth Council which is now called the Pan African Youth Parliament which works closely with the African Union.
He is the longest serving global ambassador for UNAIDS, a specialized agency of the United Nations that 

In [18]:
result

{'input': 'What are some of his famous quotes?',
 'history': [HumanMessage(content='When was Nelson Mandela born?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Unfortunately, I was unable to find the birthdate of Nelson Mandela.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='When was Nelson Mandela born?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nelson Mandela was born on July 18, 1918.', additional_kwargs={}, response_metadata={})],
 'output': 'Here are some of Nelson Mandela\'s famous quotes:\n\n1. "Education is the most powerful weapon which you can use to change the world."\n2. "I like friends who have independent minds because they tend to make you see problems from all angles."\n3. "Man\'s goodness is a flame that can be hidden but never extinguished."\n4. "The greatest glory in living lies not in never falling, but in rising every time we fall."\n5. "For to be free is not merely to cast off one\'s chains, bu

In [20]:
print(result['output'])

Here are some of Nelson Mandela's famous quotes:

1. "Education is the most powerful weapon which you can use to change the world."
2. "I like friends who have independent minds because they tend to make you see problems from all angles."
3. "Man's goodness is a flame that can be hidden but never extinguished."
4. "The greatest glory in living lies not in never falling, but in rising every time we fall."
5. "For to be free is not merely to cast off one's chains, but to live in a way that respects and enhances the freedom of others."
6. "No one is born hating another person because of the color of his skin, or his background, or his religion. People must learn to hate, and if they can learn to hate, they can be taught to love, for love comes more naturally to the human heart than its opposite."
7. "I have walked that long road to freedom. I have tried not to falter; I have made missteps along the way. But I have discovered the secret that after climbing a great hill, one only finds that